# CSci 765


# Abstract
The ongoing  COVID-19 pandemic , namely SARS-CoV-2, has created a catastrophic situation throughout the world. Its only increasing by day. Machine learning can predict the trend of the disease, predict its growth and design methods and stratagies to handle it more effectivelty. 


There is an inverse relationship between transmission and humiidty and temperature that has been known for many other post pandemics such as the spanish flu a hundred years ago.  To help predict future cases, the following four machine learning models were trained and validated: ordinary least squares linear regression, SARMIAX time series, XGBOOST, and a feed forward neural network (NN). Covid case counts and weather information for each United States county since the beginning of the pandemic in March 2020 until December 11, 2020 was imported from a a freely available data source on www.kaggle.com. The linear regression and SARIMAX models performed poorly, while the XGBOOST and NN performed very well in predicting future daily cases.

# Introduction

 Since Spanish Flu there has been no serious health hazard as the COVID‐19.Almost all the countries are facing calamitous times interms and it has been difficult to predict the trend and factors involved in the growth and spread of the disease. 


This project is inspired by a few papers on the reduced Covid-19 transmission when ambient temperature and humidity are high (Haque and Rahman 2020; Tosepu et al. 2020; Wu et al. 2020; Ma et al. 2020). These four papers describe a positive correclation between increased covid daily new cases and  decreased temperature, humidity, and high pior covid case count.

Access to accurate outbreak prediction models is essential to obtain insights into the likely spread
and consequences of infectious diseases.o reduce and track the spread of COVID-19, researchers and provider organizations have increasingly turned to artificial intelligence and machine learning tools to improve their surveillance efforts.Meteorological parameters were crucial and effective factors in past infectious diseases, like influenza and severe acute respiratory syndrome (SARS), etc. The present study targets to explore the association between the coronavirus disease 2019 (COVID-19) transmission rates and meteorological parameters. For this purpose, the meteorological parameters and COVID-19 infection data from 28th March 2020 to 22nd April 2020 of different states of India have been compiled and used in the analysis. The gradient boosting model (GBM) has been implemented to explore the effect of the minimum temperature, maximum temperature, minimum humidity, and maximum humidity on the infection count of COVID-19. 

 OLS Linear Regression
This is one of the most common statistical models taught in almost every statistics class.  It is a machine learning model because the slope and intercept are parameters that are found based on fitting to the training set.

SARIMAX Time Series
    The proper name for the model would be Seasonal Autoregressive Integrated Moving Average Exogenous model.SARIMAX is applied on data that have seasonal cycles. 
XGBOOST is a library which helps to predic scalable gradient boosting.


 Neural network (nn)
 This model predicts the patterns and they are designed after human brain.



# Literature Review

This data is from a public dataset provided here: US Counties: [US Counties: COVID19 + Weather + Socio/Health data](https://www.kaggle.com/johnjdavisiv/us-counties-covid19-weather-sociohealth-data).  It has covid daily positive cases and deaths for all 3,142 counties in the United States, along with other daily information about social, economic, health, and weather conditions in each county.

The Literature shows that there is an inverse relationship between the temprature and the covid cases.

# Concepts
Our machine learning (ML) model is a combination of ordinary least square regression, XGBoost , sarimax and  neural network. All of these machine learning models were applied to see the relationship between temprature and the covid cases. The XGboost and neural network outperformed linear regression and sarimax models.





# Experiment

The below shows the Python code for importing, cleaning, exploring, plotting and saving/retrieving the data to an SQL database, as well as training and validating the four machine learning models. See the expimental section at the end of the paper. 







# Table of Contents for Experiment
1. Import Libraries
2. Load Covid-19 Data
3. Explore and Filter Data
4. Clean and Process Data
    * Convert Dew Point and Temperature to Relative Humidity
    * Show Some Raw Data for Cass, ND
5. Function to Make Fancy Plots
    * Plotting data for Cass, ND and New York City
6. Cumulative to Daily New Cases and Deaths
    * New Plots of Cass and NYC with Daily New Cases
7. SQL Database
    * Sqlite
    * Separate DataFrame into Different Tables
    * Save Tables to SQL DB
    * Query SQL DB
    * Example Join SQL Query
8. Predict Future Covid Cases
    * Model 1: Ordinary Least Squares Linear Regression (OLS)
    * Model 2: Time Series Model (SARIMAX)
    * Model 3: XGBOOST
    * Model 4: Neural Network 
9. Comparing Machine Learning Models

# 1. Import Needed Libraries 
First import libraries needed for running the code.  

In [ ]:
# read in needed libraries
import os # accessing directory structure
import numpy as np # linear algebra
import pandas as pd # data tables with DataFrames, CSV file I/O (e.g. pd.read_csv)
from datetime import date, datetime # for handling dates and datetimes
import matplotlib.pyplot as plt # plotting and graphing

# import all the machine learning libraries!
# Helper libraries for machine learning
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Time Series Models
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
# XGBOOST model
from xgboost import XGBRegressor
# Neural Network models
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Suppress future warning messages; as a class project, not 
# concerned if a function is depricated in a few years because 
# this is not production code.
import warnings
warnings.filterwarnings('ignore')

# 2. Load Covid-19 Data

This data is from a public dataset provided here:  
[US Counties: COVID19 + Weather + Socio/Health data](https://www.kaggle.com/johnjdavisiv/us-counties-covid19-weather-sociohealth-data). 

It has covid daily positive cases and deaths for all 3,142 counties in the United States, along with other daily information about social, economic, health, and weather conditions in each county.
****
The dataset is stored on disk as a csv file. The below code reads the csv file into main memory as a pandas dataframe. This code assumes we are running on a Kaggle server because we import a csv file from a location on Kaggle.

In [ ]:
# Show all the datasets available:  there are 3 csv files in the current 
# version of the dataset. This notebook will primarily use 
# US_counties_COVID19_health_weather_data.csv.  This data needs to be 
# attached to this notebook using "Add data" before running this code.
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# During code development, only read in part of the dataset because 
# the dataset is large (>780,000 rows and >220 columns) and slowed down execution.
# Specify 'None' if want to read whole file for the full calculations
nRowsRead = None
# fname:  csv file name where data is
fname = '/kaggle/input/us-counties-covid19-weather-sociohealth-data/US_counties_COVID19_health_weather_data.csv'
# Read in file and convert into a pandas DataFrame, which is like a table.
df = pd.read_csv(fname, delimiter=',', nrows = nRowsRead)

# 3. Explore and Filter Dataset

In [ ]:
# change date (which is now a string) into a datetime object
df['date'] = df['date'].astype('datetime64')


nRow, nCol = df.shape
print(f'The full dataset for all counties in the USA has {nRow} rows and {nCol} columns\n')

num_counties = len(df['county'].unique())
num_states   = len(df['state'].unique())
first_date = df['date'].min()
last_date  = df['date'].max()

print('More statistics on entire dataset for all counties and dates.')
print(f'Number of counties in dataset: {num_counties}')
print(f'Number of states in dataset:   {num_states}')
print(f'First date of data:            {first_date}')
print(f'Last date of data:             {last_date}')

# LIMIT THE DATA USED IN THE PROJECT TO JUST NEW YORK STATE AND NORTH DAKOTA
# This allows the code to run much faster. This reduces the number of rows by
# more than 96%
df = df[ (df['state']=='North Dakota') | (df['state']=='New York') ]
nRow, nCol = df.shape
print(f'\nThe dataset limited to New York State and North Dakota has {nRow} rows and {nCol} columns')

The table has over 200 columns of information about cases, deaths, county information, stay at home orders, geographic position of the county, socioeconomic data by county, weather data by county, etc.  See a more detailed description of the columns from the source page of this data [here](https://www.kaggle.com/johnjdavisiv/us-counties-covid19-weather-sociohealth-data). Below prints out a list of all columns, then eliminates most columns to focus on a small subset of columns used for this project. Many of the columns that were eliminated would likely be helpful in predicting future covid deaths. But the number of columns was limited to keep the scope of the project managable.

In [ ]:
print(f'First 20 of over 200 columns names in table: \n{df.columns.values[0:20]}') 

keep_columns = \
['date', 'county', 'state', 'fips', 'cases', 'deaths', 'lat', 'lon', 'total_population', 
'population_density_per_sqmi', 'mean_temp','mean_temp_5d_avg', 'mean_temp_15d_avg', 
'dewpoint', 'dewpoint_5d_avg', 'dewpoint_15d_avg']

df = df[ keep_columns ] # keep only the above columns

Let's take a look at the first 5 rows of the table just read in.  The table has one row of data for each date for each US county since officials started keeping track of deaths begining on January 21, 2020. The cases column represents the cumulative number of positive cases in the county as of the given date.  Similarly for deaths.  

In [ ]:
df.head(5) # show first five rows of DataFrame

 # 4. Clean and Process Data
 
 The dataset is already mostly cleaned.  There are a few things to do, like converting strings of dates into datatime objects and removing rows with mising values (NaN).  Data for a number of US Teritories was removed because of missing values.  Even after this removal, there are still some missing values for some temperatures and dew points.

In [ ]:
# Check if table has missing data (NaN) before remove any rows
missing = df.isna().sum() # count of number of missing values for each column
print('Count of number of missing values in each column of DataFrame')
print(missing)

print('\nHow many rows have missing mean_temp values grouped by state?')
print(df[ df['mean_temp'].isna() ].groupby(['state']).count()['date'])

The above shows there are missing values in New York and North Dakota in the temperature and dewpoint data. Each county may have some of these values missing. The rest of the notebook focuses 2 counties: Cass, North Dakota and New York City.  These two cities have few missing values. Rows with missing values will be eliminated later because missing values often cause problems for the machine learning algorithms.

Alternatively, the missing weather data can be estimate for certain counties based on the temperature and dewpoint of the county closest which has those values.  That calculation is not done because it is more complicated to calculate the Euclidian distance between all counties based on the each county's latitude and longitude in the table. It could be done for an improved version of this analysis in the future.

In [ ]:
df_geo = df.drop_duplicates(subset=['county', 'state'], keep='first')
df_geo = df_geo[ ['county', 'state', 'lat', 'lon'] ]
print('Table of County Latitude and Longitude')
print(df_geo.head())
# TODO:  calculate distance between all counties.
# Will not do this now because developing this code will take too long
# In the mean time, just ignore the counties with missing weather values.

### Convert Dew Point and Temperature to Relative Humidity

The dataset gives the daily temperature and dew point for each county. The dew point is the temperature below which water droplets begin to condense and dew can form.  The values of interest are the temperature and relative humidity. The below function calculates the relative humidity from the temperature and dew point using the August-Roche-Magnus approximation, which was converted into a python function based on this [website](https://bmcnoldy.rsmas.miami.edu/Humidity.html). 

Reference: Alduchov, O. A., and R. E. Eskridge, 1996: Improved Magnus' form approximation of saturation vapor pressure. J. Appl. Meteor., 35, 601–609.

In [ ]:
 def relative_humidity(t, dp):
    '''Calculate relative humidity from the temperature and dew_point
    Values are calculated using the August-Roche-Magnus approximation.
    
    Parameters
    ------------
    t : temperature in Fahrenheit
    dp : dew point in Fahrenheit
    
    Returns
    ------------
    float
        Relative humidity
    '''   
    # Equation from reference takes temperature in celcius.
    # First convert temperatures to celcius.
    t  = (t  - 32) * (5/9) 
    dp = (dp - 32) * (5/9) 
    return 100*(np.exp((17.625*dp)/(243.04+dp))/np.exp((17.625*t)/(243.04+t)))

# convert dew point column to relative humidity
# The below apply method is advanced:  it goes row by row in the table,
# and passes the mean_temp value and dewpoint value into the relative_humidity
# function defined above, then assigns that relative humidity value to a 
# new relative_humidity column.  The axis=1 means that the apply function
# runs row-by-row, instead of the default behavior of applying the operation
# to the entire column. Because the relative_humidity function needs both 
# temperature and dewpoint, which are in two different columns, the operation
# needs to be done row by row.  See:
# https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
df['relative_humidity'] = df.apply(
    lambda x: relative_humidity(x['mean_temp'], x['dewpoint']), axis=1)
df['relative_humidity_5d_avg'] = df.apply(
    lambda x: relative_humidity(x['mean_temp_5d_avg'], x['dewpoint_5d_avg']), axis=1)
df['relative_humidity_15d_avg'] = df.apply(
    lambda x: relative_humidity(x['mean_temp_15d_avg'], x['dewpoint_15d_avg']), axis=1)

# drop the dewpoint columns because they are no longer needed after the humidity was calculated
df.drop(['dewpoint', 'dewpoint_5d_avg', 'dewpoint_15d_avg'], inplace=True, axis=1)

### Plot histogram of Humidity
Just for fun plot a histogram of the relative humidity that was just calculated above. The plot shows a histogram for each month of humidity data for all days for all counties.  This plot is not is a sanity check to make sure all the relative humidities are between 0% and 100%.  It also shows what is possible with python and the matplotlib plotting library.

In [ ]:
# view a histogram of all the relative humidities for every day for every county
df['relative_humidity'].hist() # look only at relative_humidity column, and get histogram
# Next few lines of code set the chart title and axis labels.
plt.title('Histogram of Relative Daily Humidity for All US Counties')
plt.ylabel('Count')
plt.xlabel('Relative Humidity %')
plt.show()

### Show Some Raw Data for Cass, North Dakota
Showing this because this is home.

In [ ]:
# number of days of data for Cass, North Dakota
only_cass = (df['county']=='Cass') & (df['state']=='North Dakota')
print( f'Number of days of data for Cass, North Dakota: {sum(only_cass)}')
df_cass = df[ only_cass ]
df_cass.head()


# 5. Function to Make Fancy Plots

The below function makes fancy plots of cases and deaths for a single county in a particular state. Temperature and humidity are also plotted. [This tutorial](https://www.tutorialspoint.com/matplotlib/index.htm) on matplotlib plotting library was used to understand how to make these plots****. This function does a lot of fancy things, like plotting two side by side plots with twin axes, as well as adding legends and configuring the tick marks and tick labels.

In [ ]:
def plot_county(df, county, state, start=None, end=None):
    '''Plots the confirmed cases and deaths for a single county in a 
    a particular state, as well as temparature and humidity.
    
    Parameters
    ------------
    df : pandas DataFrame with data
    county : county to plot;  e.g., 'Cass'
    state: state where county is located; e.g., 'North Dakota'
    start: start date to begin plot; if None, plot all dates in df
    end: end date to stop plotting; if None, plot all dates in df
    '''
    one_county = (df['county']==county) & (df['state']==state)
    df_county = df[ one_county ]
    if start: # if the start is NOT None
        if type(start) == str:
            # converts string of date into date; i.e, "2020-04-11" string becomes a date
            start = parse(start).date() 
        only_dates = df_county['date'] >= start
        df_county = df_county[ only_dates ]
    if end: # if the end date is NOT None
        if type(end) == str:
            # converts string of date into date
            end = parse(end).date() 
        only_dates = df_county['date'] <= end
        df_county = df_county[ only_dates ]
    
    # Create a figure with two side by side chart
    # ax1 is the left hand chart; ax2 is the right hand chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    # Left chart of Date vs. Covid Cases
    ax1.plot( # plot function to graph points and lines 
        df_county['date'], # x values 
        df_county['cases'],# y values 
        'bo--', # format: 'b'=blue 'o'=solid circles '--'=dashed line
        label='Cases') # label of this line for the legend     
    ax1.set_title(f"{county}, {state} Confirmed Cases")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("ConfirmedCases")
    # create a second plot within this graph with second y-axis on right
    ax1_t = ax1.twinx()  
    ax1_t.plot( # plot function to graph points and lines 
        df_county['date'], # x values 
        df_county['mean_temp_5d_avg'], # y values 
        'ro--', # format: 'r'=red 'o'=solid circles '--'=dashed line
        label='Temp') # label of this line for the legend     
    ax1_t.set_ylabel("Temperature 5 Day Avg")
    # limit number of x-ticks to 11, otherwise have 100+ dates that are written over each other
    ax1.xaxis.set_major_locator(plt.MaxNLocator(11)) 
    # rotate text of x-axis tick labels 45 degrees
    ax1.tick_params(axis='x', labelrotation=45)    
    # Add legends: one for first plot (ax1), and for 2nd plot (ax1_t)
    # loc=2 means put legend in upper left corner
    ax1.legend(loc=2) # add legend for first plot
    # bbox_to_anchor shifts down a little so the two legends are not on top of each other
    ax1_t.legend(loc=2, bbox_to_anchor=(0.0, 0.95)) 
    
    # same as above, but for chart on right side
    ax2.plot(df_county['date'], df_county['deaths'], 'bo--', label="Deaths")
    ax2.set_title(f"{county}, {state} Deaths")
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Deaths")
    ax2_t = ax2.twinx()  
    ax2_t.plot(df_county['date'], df_county['relative_humidity_5d_avg'], 'ro--', label='Humidity')  
    ax2_t.set_ylabel("Relative Humidity 5 Day Avg")
    ax2.xaxis.set_major_locator(plt.MaxNLocator(11))
    ax2.tick_params(axis='x', labelrotation=45)
    ax2.legend(loc=2) 
    ax2_t.legend(loc=2, bbox_to_anchor=(0.0, 0.95)) 

    # improves separation between graphs so axis don't overlap
    fig.tight_layout() 
    
    plt.show() # display the plots to the screen

## Plots of Cass, ND and New York City 

Note:  deaths and cases are cumulative, not daily new deaths and cases.

In [ ]:
# run the function to plot for a particular county and state
plot_county(df,"New York City","New York") # start='2020-10-01', end='2020-10-31')
# run the function to plot for another county and state
plot_county(df,"Cass","North Dakota")


# 6. Cumulative to Daily Cases and Deaths

The above data on deaths and positive cases shows the cumulative amounts up to the given dates. It's better to predict the daily number of new cases and new deaths. The pandas DataFrame function diff() takes the difference between the current row in the table and the previous row. The df DataFrame is one giant table of all counties for all days. Taking the difference in df would mix up counties. First the main df needs to be grouped by county, then the daily difference taken separately for each county.

In [ ]:
# Group by county and state. Then for each group (i.e., each separate county,state)
# Take the difference from the current day cases from the previous day
# Do the same for deaths. Note: name is not used, but it contains the name
# of the group, which is just the county,state; need to have it there because
# of how iterating through groupby works.
df_new = pd.DataFrame() # new dataframe to store the changed data
for name, g in df.groupby(['county', 'state']): 
    # g is the DataFrame for one county,state
    # save cumulative cases and deaths in a new column
    g['cases_cumulative'] = g['cases'] 
    g['deaths_cumulative'] = g['deaths']
    # overwrite cases and deaths column with daily new cases and new deaths
    g['cases']  = g['cases'].diff()
    g['deaths'] = g['deaths'].diff()
    # append this DataFrame for one county into a single DataFrame for all counties
    df_new = pd.concat([df_new, g])

# overwrite the old dataframe df with the new one that has daily deaths and cases
df = df_new

### New Plots of Cass and NYC with Daily New Cases and Deaths

Run the function to plot for a particular county and state, but now the data is for daily new deaths and new cases. You can clearly see the trend with temperature: as the average temperatures increased during the summer, the number of new cases went down.

In [ ]:
plot_county(df,"New York City","New York") 
plot_county(df,"Cass","North Dakota")

In [ ]:
# show first 10 rows of new dataframe.
# The rows are now in order of counties. The index (far left column)
# is still the row number from the original table. The rows are reordered by
# county now, so those index numbers are out of order from the original table.
df.head(10)

# 7. SQL Database

This section shows how to store the above data into an SQL database, then retrieve data through SQL queries to the database. Originally, the plan was to use Google BigQuery, but the setup time would have been excessive. Accessing the postgreSQL database setup at the university was tried, but the credentials no longer worked. Instead, sqlite was used. Sqlite is included in the python distribution. It creates and sets up a database file on the local disk (wherever this python code is run). This setup can be easily changed to save and query the postgreSQL server at the university by changing the connection variable, con.  

Technically, saving to an SQL db and querying that db is not necessary for the machine learning predictions in this project because the full dataset was available on Kaggle as a csv file that can be handled in main memory (it's about 1 GB). This section is done as a requirement for the class to show how to do some database operations from within python. The workflow is to save tables from pandas dataframes to the SQL database, then query that database and save the query results to a pandas dataframe.

In [ ]:
# Attempt to connect to NDSU school postgreSQL database
# FAILED.  Could not get connection to work.  

# before use the library psycopg2, it needs to be installed
# because the server on Kaggle does not include it
#!pip install psycopg2 ## does not work because can't build from source
# must install binary
#!pip install psycopg2-binary

# previously worked with sqlite.  Now do it in
# postgreSQL, connected to DB server hosted by University
# import psycopg2 # library to connect to postgreSQL 
# con=psycopg2.connect(
#     host='shinji.cs.ndsu.nodak.edu',
#     user='aemanuel_765f20',
#     
#     
#     password='6L6WAd6y',   
#     dbname='aemanuel_db765f20' 
# )
# Had errors with establishing connection to server
# TODO: debug
# Showing code above because once the credentials work and the 
# database is properly running on the school server, the below 
# code will work, and all the next steps are the same as for 
# sqlite. In the mean time, continuing with sqlite db on this server
# The commands should be identical to what would do in postgreSQL
################################################################

### SQLITE

In [ ]:
# Store the data in a relational database
# sqlite is a real SQL database 
import sqlite3 # sqlite library in python

# create a connection to a database
con = sqlite3.connect("covid.sqlite")

# will throw error if that table is not there
# only run if the table was there already (done during code development)
#con.execute("DROP TABLE usa_county_covid") 
#con.execute("DROP TABLE fips_table") 

### Separate DataFrame into Different Tables

The table of data (df) that was read in from the Kaggle dataset on Covid and then processed above is one massive table with many columns of information. It could be saved as is to an SQL table, but that would not be a good excersize in relational databases. To simulate a relation between different tables, the main DataFrame is separated into two tables: one with county information, indexed by fips (df_fips), and another with the daily death and cases (df_daily).  fips is a unique code, like a zip code, for each county in the USA. The original df had repeated information in the county, state, lat and lon columns. The same county and state name, as well as latitute and longitude was repeated for every day of data for that county.  Instead, it's better to put this in a separate table which can be accessed by the unique fips code for each county.  The main df_daily contains the fips code to reference the county and state name, but it should not have the county and state name in it.  The below code does that.

In [ ]:
# fips to county and state info, as well as latitude and longitude
df_fips = df[ ['fips', 'county', 'state', 'lat', 'lon'] ]  
# drop the county, state, lat, lon columns from the main dataframe df
# because this columns will be in a separate table you can lookup via fips code
df_daily = df.drop(['county', 'state', 'lat', 'lon'], axis=1)

# df_fips has many duplicates because each county has a row for EVERY day of COVID
# only want one row for each county
df_fips = df_fips.drop_duplicates()

# make the fips code the index in the dataframe
# the index of a dataframe becomes the primary key when you send this to SQL
df_fips.set_index('fips', inplace=True)

### Save Tables to SQL DB

In [ ]:
# save the fips table to sqlite table
df_fips.to_sql(name='fips_table', con=con)
# save the main table to sqlite 
df_daily.to_sql(name='usa_county_covid', con=con)

In [ ]:
# print the working directory (pwd unix command)
# This is the directory on the computer where this python code is running
%pwd 

In [ ]:
# lists the files in that directory, and see that covid.sqlite db file is there
# The 'covid.sqlite' is the file that was created in the above commands that save
# information to the SQL database, which is saved as this file.
%ls /kaggle/working 

### Query SQL DB

In [ ]:
# Query the SQL db to show all the names of tables in the db
# This shows that the two tables stored above are correctly in the db.
# for some variages of SQL db's, you just write "SHOW TABLES"
# sqlite has a more complicated command
# this shows all the tables stored in the covid.sqlite database
con.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [ ]:
# Query the fips_table to get rows on Cass, North Dakota
con.execute("SELECT * FROM fips_table WHERE County='Cass' AND State='North Dakota'").fetchall()

In [ ]:
# Using the fips code for Cass, ND found in the above query, run a new
# query on the main usa_county_covid table limiting what's returned to that fips code
# This table can not have a WHERE clause for the county and state because
# those columns are NOT in this table.
r = con.execute("SELECT * FROM usa_county_covid WHERE fips=38017")
list_of_rows = r.fetchall()
# demonstration of looping through the list of all returned rows, row by row
# [0:5] limits the printing to the first 3 rows so don't have a ton of output 
# for all days
for row in list_of_rows[0:3]:
    print(row)

In [ ]:
# Check that the data returned from SQL is the same as the original
# dataframe. The below code does the same thing as the sql above, but uses 
# the original dataframe;  head(3) only prints out the first 3 rows.
df[ df['fips']=='38017' ].head(3) 

### Example JOIN SQL Query

The above code has two separate queries: one to the fips_table to lookup the fips code for Cass, ND, then a second query using that code to get daily data for Cass. The below query runs a join to do both those things at the same time in one query.

In [ ]:
# JOIN query 
list_of_rows = con.execute(''' 
SELECT * FROM usa_county_covid
INNER JOIN fips_table ON fips_table.fips=usa_county_covid.fips
WHERE fips_table.county="Cass" AND fips_table.state="North Dakota"
''').fetchall()
print(f'number of rows returned: {len(list_of_rows)}')
# print out first 3 rows returned
print(list_of_rows[0:3])
                                  

In [ ]:
# Show the column names and data types for one of the tables
# This is code for sqlite. Other SQL versions may be slightly different.
con.execute('PRAGMA table_info(usa_county_covid);').fetchall()

In [ ]:
# If all the data had been originally in an SQL database as two s
# eparate tables, here is how to bring all the data into a joined 
# table into a pandas dataframe.  Once in the dataframe, it can be 
# used to do the rest of the data science and machine learning
list_of_rows = con.execute(''' 
SELECT * FROM usa_county_covid
INNER JOIN fips_table ON fips_table.fips=usa_county_covid.fips 
''').fetchall()
print(f'number of rows returned: {len(list_of_rows)}')
# print out first two rows returned
for r in list_of_rows[0:3]:
    print(r)

In [ ]:
# Same as above, but save query directly into pandas DataFrame
df_fromSQL = pd.read_sql('''
SELECT * FROM usa_county_covid
INNER JOIN fips_table ON fips_table.fips=usa_county_covid.fips
''', con=con)
# show the dataframe that was just created from the SQL query
df_fromSQL.head(3)

# 8. Predict future Covid cases
1. Model 1:  Ordinary Least Squares Linear Regression (OLS)
2. Model 2:  Time Series Model (SARIMAX)
3. Model 3:  XGBOOST
4. Model 4:  Neural Network

# Model 1: OLS Regression

Ordinary Least Squares Regression is used here to predict covid cases. This is a very common machine learning regression. It is taught in almost every statistics class.  It can predict linear trends from the data using multiple columns of features.  A limitation of this model is that it is linear and can't do seasonal or non-linear forcasting. The number of new Covid cases is seasonal - meaning that it goes up and down depending on the date. In the winter during cold months, cases go up, while cases go down during the warm months. This may happen because during cold months, people congregate inside which spreads the disease more.  

Linear regression, as seen in the below chart, is a poor model for seasonal data like Covid cases. Below is the work showing how to do a linear regression model, and a plot of the predictions.  

### **Only New York City** is used for the rest of this project simplicity. Predictions for other counties could be done by changing one line of code below.

Also, only the date is used as the independent variable, rather than using all the other data like temperature, humidity, etc. because it is known that linear regression doesn't work well. Adding more features (i.e., columns) for a multiple linear regression may slightly improve the model, but it will still be horrible.

In [ ]:
# Test:Linear regression for one county
#df1 = df[ df['fips']=='38017' ] # only Cass, North Dakota from fips code
#####################################################################
#### CHANGE THE BELOW LINE OF CODE TO PREDICT FOR OTHER COUNTIES ####
#####################################################################
df1 = df[ (df['county']=='New York City') & (df['state']=='New York') ]
df1 = df1.copy() # make a proper separate copy of this dataframe

# Recall that cases column is the difference from the previos day,
# so the first row has no previous day, and that value became NaN (missing)
df1 = df1[1:]    # remove the 1st row, which has NaN for cases

# reset the index; previously the index was the row number from the original
# huge table of all counties. The row number for Cass county was mixed in with 
# rows from other counties. Reset to have the index start at 0 and go up by 1 for
# each day for only Cass county rows.
df1.reset_index(inplace=True)

In [ ]:
# practice to get the commands correct in calculating number of days between two dates
aDate = df1['date'][0] # taking just a single date as an example from the dataframe
# calculating the number of days between two dates:
startDate = datetime(2020, 3, 1)
day_diff = aDate - startDate # this returns a timedelta object
num_days = day_diff.days # returns the number of days out of the timedetla object
print(f'Difference between dates \n{startDate}  \n{aDate}\nis {num_days} days')

In [ ]:
# calculate difference in days between March 1, 2020 and the date of each row
df1['days'] = df1['date'].apply(lambda x: (x - datetime(2020,3,1) ).days )

Limit the input data to just the days. 

In [ ]:
# the target variable; i.e., what it being predicted; the dependent variable
y = df1['cases'] 
# X the features; i.e., the factors or the independent variables
X = df1[ ['days']]

In [ ]:
# To run a multiple linear regression on more features, do
# something like this; Not done for simplicity
#df1 = df1[ ['days', 'cases', 'mean_temp_5d_avg', 'relative_humidity_5d_avg'] ]
# remove any rows with missing values because they cause errors with linear regression
#df1 = df1.dropna()
#X = df1[ ['days', 'mean_temp_5d_avg', 'relative_humidity_5d_avg']]

Create and fit the linear regression model.  

In [ ]:
from sklearn import linear_model # library with OLS regression
model = linear_model.LinearRegression()  # define the linear regression model
# RUN THE LINEAR REGRESSION; i.e., fit the model to the data
model.fit(X,y)

### Create Plot
Show the linear regression vs. the real data.

In [ ]:
# Create plot showing the linear regression vs. the real data
# Create 200 data points between 0 ~ 280 that are equally spaced
# This is used to create the nice line in the plot of the linear regression
x_pred = np.linspace(0, 280, 200)  
# preprocessing required by scikit-learn functions
x_pred = x_pred.reshape(-1, X.shape[1])  
# Use the equally spaced line to generate predictions 
# from the model created above. This makes the y values for drawing the line
y_pred = model.predict(x_pred)

# Create a figure (fig) of size 9" wide and 5" tall with one plot (ax) in it.
fig, ax = plt.subplots(figsize=(9,5))

# Add the linear regression line to the plot ax 
ax.plot(x_pred, y_pred, color='k', label='Linear Regression Model')
# Add the real data to the plot as circles with red boarders and grey interiors 
# that are partially transparent (alpha=0.5) so overlapping circles show.
ax.scatter(X, y, edgecolor='r', facecolor='grey', alpha=0.5, label='Real Data')
# Add Axes labels and Plot title
ax.set_ylabel('Number of New Daily Cases', fontsize=14)
ax.set_xlabel('Days since March 1, 2020', fontsize=14)
ax.set_title('Linear Regression of Covid Cases for New York City')
ax.legend(facecolor='white', fontsize=11)
# Add the predicted Linear Regression equation to the plot 
ax.text(0.45, # location to write text in x direction (45% from left)
        0.35, # location to write text in y direction (35% from bottom)
        # Print out the coefficients as an equation
        '$y = %.2f x + %.2f $' % (model.coef_[0], model.intercept_), 
        fontsize=17, 
        transform=ax.transAxes # needed for above 45%, 35% positioning to work
       )
plt.show()

As shown in the above plot, the straight line of the linear regression does not do well with seasonal data.

# Model 2: Time Series - SARIMAX

SARIMAX stands for Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors model. A description of how this complicated model works is beyond the scope of this project. A major improvement over the linear regression is that time series models like SARIMAX are designed for seasonal data like what is here with the covid cases.

This machine learning technique as well as the next few sections are based on the Coursera course: [Compare time series predictions of COVID-19 deaths](https://www.coursera.org/projects/compare-time-series-predictions-of-covid19-deaths). This project adds more data and also adds more features to the later models beyond solely the date. 

In [ ]:
#print(df1.to_string())
#print(df1.columns)
df2 = df1[ ['date', 'cases'] ]
print(f'Are there any missing values?')
print(df2.isna().sum()) # print number of missing values in each column
df2 = df2.dropna()      # remove rows with missing values
#X = df2['date']         # features; i.e., independent variables. Only one here, the date
#y = df2['cases']        # predicted variable; dependent variable, the number of cases
df2 = df2.set_index('date')  # make the date the index of the DataFrame, which is needed in time series


Now let's talk about SARIMAX. There are three hyperparameters that go into the order tuple: p, q and d. Here we've used p=2, q=1 and d=3.

In [ ]:
# Split dataset into training and test sets before and after Nov. 15, 2020
# The model will only learn from data before Nov. 20, 2020
# and then it will be tested if it can predict the data after Nov. 15, 2020
start_date1 = datetime(2020, 3, 3)
start_date2 = datetime(2020, 8, 1)
split_date  = datetime(2020,11,15)
train1 = df2[ (df2.index <  split_date) & (df2.index > start_date1)  ]
train2 = df2[ (df2.index <  split_date) & (df2.index > start_date2)  ]
test  = df2[ df2.index >= split_date  ]

In [ ]:
# Create 2 SARIMAX models: 
# model1 that uses data from March 3, 2020 on for prediction
#  this model has problems because SARIMAX gets confused with the big spike 
#  in April, then another spike starting in October
# model2 that uses data only from Aug. 1, 2020 on for prediction
# completely ignoring prior data. It then gets the upward trend correct
# for the 2nd wave. Of course linear regression would also do better 
# under these conditions, too.
model1 = SARIMAX(train1, order=(2,1,3))
results1 = model1.fit(disp=True)

model2 = SARIMAX(train2, order=(2,1,3))
results2 = model2.fit(disp=True)

In [ ]:
# Plot the SARMIA predictions vs. the real data
sarimax_prediction1 = results1.predict(
    start=split_date, end='2020-12-20', dynamic=False)
sarimax_prediction2 = results2.predict(
    start=split_date, end='2020-12-20', dynamic=False)
plt.figure(figsize=(9,7))
l1, = plt.plot(df2, label='Observation')
l2, = plt.plot(sarimax_prediction1, label='SARIMAX model 1')
l3, = plt.plot(sarimax_prediction2, label='SARIMAX model 2')
plt.legend(handles=[l1, l2, l3])
plt.ylim([0,6000]) # adjust the y-axis lowest and highest value.
plt.savefig('SARIMAX prediction', bbox_inches='tight', transparent=False)

The two SARIMAX models shown above.  Model1 which uses all prior data for prediction is poor. It is a flat line because the model does not do well with with a single wave 1 case spike in April, then another spike starting in October.  ARIMA models need to observe multiple waves to tune the seasonality.  We only have one full wave and are in the increasing part of the 2nd wave.  Model 2 adjust the training set to include only data from August onwards, totally ignoring the data from the first wave of covid cases. The model performs better because it get the general upward trend of cases now and is not confused by the previous spike. Both models are poor because of limitations or tweeking the input data.


# Model 3: XGBoost

Unlike the prophet and SARIMAX models, XGBOOST (as well as the next neural network model done last), are supervised machine learning models that deal with independent data points. It assumes that each data point is totally independent from the rest of the data points in the dataset. The SARIMAX does not assume independence because it treats it like a time series - previous days data help predict the future.  Obviously the number of cases from the previous days helps predict the cases in the coming days. i.e., if there are very few cases, there will likely be very few cases the next few days because there are few people sick to spread the disease. So the previous data needs to be linked to the current data.  The featurize method does that, by adding columns of data that link the current row of data to past dates. 


In [ ]:
df2 = df1
df2 = df2.copy()
df2 = df2.set_index('date')
df2 = df2.drop(['fips', 'index', 'county', 'state', 'deaths', 
                'lat', 'lon', 'total_population', 'days',
                'cases_cumulative', 'deaths_cumulative'], 
               axis=1)

In [ ]:
def featurize(t):
    X = pd.DataFrame()

    X['day']        = t.index.day
    X['month']      = t.index.month
    X['quarter']    = t.index.quarter
    X['dayofweek']  = t.index.dayofweek
    X['dayofyear']  = t.index.dayofyear
    X['weekofyear'] = t.index.weekofyear
    
    X.index = t.index
    for c in t.columns:
        if c=='case': 
            next
        X[c] = t[c]
            
    y = t.cases
    return X, y

df2 = df2.dropna()
print('Are there any missing values that can mess up the algorithms?')
print(df2.isna().sum())

featurize(df2)[0].head()

In [ ]:
X_train, y_train = featurize(
    df2.loc[df2.index <  split_date])
X_test, y_test = featurize(
    df2.loc[df2.index >= split_date])

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
scaled_train = scaler.transform(X_train)
scaled_test  = scaler.transform(X_test)

In [ ]:
XGBOOST_model = XGBRegressor(n_estimators=30)
XGBOOST_model.fit(scaled_train, y_train, 
                  eval_set=[(scaled_train, y_train), (scaled_test, y_test)],
                  verbose=True)
XGBOOST_prediction = XGBOOST_model.predict(scaled_test)
print('XGBOOST MAE = ', mean_absolute_error(XGBOOST_prediction, y_test))

# Model 4: Neural Network

In [ ]:
NN_model = Sequential()
NN_model.add(Dense(20, input_shape=(scaled_train.shape[1], )))
NN_model.add(Dense(10))
NN_model.add(Dense(1))
NN_model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.001))
NN_model.fit(scaled_train, y_train, validation_data=(
    scaled_test, y_test), epochs=220, verbose=1)
NN_prediction = NN_model.predict(scaled_test)

# 9. Comparing Machine Learning Models

In [ ]:
#print('SARIMAX MAE = ', mean_absolute_error(sarimax_prediction, y_test)) # TODO fix code.
print('XGBOOST MAE = ', mean_absolute_error(XGBOOST_prediction, y_test))
print('NN MAE      = ', mean_absolute_error(NN_prediction, y_test))

In [ ]:

XGBOOST_df = pd.DataFrame({'y': XGBOOST_prediction.tolist()})
XGBOOST_df.index = y_test.index

NN_df = pd.DataFrame(NN_prediction)
NN_df.index = y_test.index
fig, axs = plt.subplots(2, 2, figsize=(10,10))
fig.suptitle('Compare Linear Regression, SARIMAX, XGBOOST and NN')

y = df1['cases'] 
# X the features; i.e., the factors or the independent variables
X = df1[ ['days']]
axs[0, 0].plot(df1['days'].tail(50), df1['cases'].tail(50))
axs[0, 0].plot(x_pred[-30:], y_pred[-30:])
axs[0, 0].set_title("Linear Regression")

axs[0, 1].plot(df2['cases'].tail(50))
axs[0, 1].plot(sarimax_prediction1.tail(50))
axs[0, 1].set_title("SARIMAX")

axs[1, 0].plot(df2['cases'].tail(50))
axs[1, 0].plot(XGBOOST_df.tail(50))
axs[1, 0].set_title("XGBOOST")
axs[1, 1].plot(df2['cases'].tail(50))
axs[1, 1].plot(NN_df.tail(50))
axs[1, 1].set_title("NN")

for ax in fig.get_axes():
    ax.label_outer()
fig.autofmt_xdate()

#plt.tight_layout()
plt.show()

The above plots show the real data in blue and the machine learning predictions in orange.  The data is for New York City only.

# 10. Conclusion

### XGBOOST followed by NN (neural network) has best predictions!  
### Linear regression and SARIMAX perform poorly.

XGBOOST and the NN are able to accurately predict the number of new covid cases for New York City for 20 days into the future from November 15 to December 4 (the dates in the test set) based on a training set with data from the beginning of March until November 15, 2020.  Those two algorithms use the below features to make predictions of the number of new daily cases.  

Linear regression used many features, too, but was limited by forcing the predicted trend to be a staight line (the definition of linear regression).  

SARIMAX could not pick up the wider trend in the data because of two reasons:  (1) it did not have enough examples of multiple infection waves to see seasonality of the waves - there were only two waves of infection in the pandemic. (2) it only has one feature - number of days into the pandemic, and ignores other data such as temprerature, humidity, population density, etc.

In [ ]:
print('Features used for prediction in XGBOOST and NN:')
print(X_train.columns.values)

# 11. Future Research

1. Run predictions for multiple counties to evaluate the various models for more than just New York City.  This would be a more comprehensive evaluation.  
2. Show the mean squared error rates of the predictions.  
3. Verify there are no feedback loops coming from the 5 and 15 day average temperatures and humidity data. Those are averages which "see" into the future because the average for the 15 day temperature, for example, is the average from the past 7 days and future 7 days. If this feedback into the future is predictive, is like the algorithm is cheeting by being able to see 7 days into the future.  Not sure if this is happening.

# Bibliography
Alduchov, Oleg A., and Robert E. Eskridge. 1996. “Improved Magnus Form Approximation of Saturation Vapor Pressure.” Journal of Applied Meteorology and Climatology 35 (4): 601–9. https://doi.org/10.1175/1520-0450(1996)035<0601:IMFAOS>2.0.CO;2.

Chaurasia, Vikas, and Saurabh Pal. 2020. “Application of Machine Learning Time Series Analysis for Prediction COVID-19 Pandemic.” Research on Biomedical Engineering, October. https://doi.org/10.1007/s42600-020-00105-4.

Haque, Syed Emdadul, and Mosiur Rahman. 2020. “Association between Temperature, Humidity, and COVID-19 Outbreaks in Bangladesh.” Environmental Science & Policy 114 (December): 253–55. https://doi.org/10.1016/j.envsci.2020.08.012.

Ketkar, Nikhil. 2017. “Introduction to PyTorch.” In Deep Learning with Python: A Hands-on Introduction, edited by Nikhil Ketkar, 195–208. Berkeley, CA: Apress. https://doi.org/10.1007/978-1-4842-2766-4_12.

Ma, Yueling, Yadong Zhao, Jiangtao Liu, Xiaotao He, Bo Wang, Shihua Fu, Jun Yan, Jingping Niu, Ji Zhou, and Bin Luo. 2020. “Effects of Temperature Variation and Humidity on the Death of COVID-19 in Wuhan, China.” Science of The Total Environment 724 (July): 138226. https://doi.org/10.1016/j.scitotenv.2020.138226.

McKinney, Wes. 2012. Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython. O’Reilly Media, Inc.

Suzuki, Yoshiro, Ayaka Suzuki, Shun Nakamura, Toshiko Ishikawa, and Akira Kinoshita. 2020. “Machine Learning Model Estimating Number of COVID-19 Infection Cases over Coming 24 Days in Every Province of South Korea (XGBoost and MultiOutputRegressor).” MedRxiv, May, 2020.05.10.20097527. https://doi.org/10.1101/2020.05.10.20097527.

Tosepu, Ramadhan, Joko Gunawan, Devi Savitri Effendy, La Ode Ali Imran Ahmad, Hariati Lestari, Hartati Bahar, and Pitrah Asfian. 2020. “Correlation between Weather and Covid-19 Pandemic in Jakarta, Indonesia.” Science of The Total Environment 725 (July): 138436. https://doi.org/10.1016/j.scitotenv.2020.138436.

Wang, Jingyuan, Ke Tang, Kai Feng, Xin Lin, Weifeng Lv, Kun Chen, and Fei Wang. 2020. “High Temperature and High Humidity Reduce the Transmission of COVID-19.” SSRN Scholarly Paper ID 3551767. Rochester, NY: Social Science Research Network. https://doi.org/10.2139/ssrn.3551767.

Wieczorek, Michał, Jakub Siłka, and Marcin Woźniak. 2020. “Neural Network Powered COVID-19 Spread Forecasting Model.” Chaos, Solitons & Fractals 140 (November): 110203. https://doi.org/10.1016/j.chaos.2020.110203.

Wu, Yu, Wenzhan Jing, Jue Liu, Qiuyue Ma, Jie Yuan, Yaping Wang, Min Du, and Min Liu. 2020. “Effects of Temperature and Humidity on the Daily New Cases and New Deaths of COVID-19 in 166 Countries.” Science of The Total Environment 729 (August): 139051. https://doi.org/10.1016/j.scitotenv.2020.139051.
